In [1]:
%pip install cmem-cmempy llama-index python-dotenv llama-index-vector-stores-postgres

Note: you may need to restart the kernel to use updated packages.


In [2]:
%load_ext dotenv
%dotenv
%reload_ext dotenv

In [3]:
import os

import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

In [4]:
!docker pull ankane/pgvector
!docker run -d -e POSTGRES_USER=eccenca -e POSTGRES_PASSWORD=eccenca -e POSTGRES_DB=eccenca --name pgvector-eccenca -p 5432:5432 ankane/pgvector

Using default tag: latest
latest: Pulling from ankane/pgvector
Digest: sha256:956744bd14e9cbdf639c61c2a2a7c7c2c48a9c8cdd42f7de4ac034f4e96b90f8
Status: Image is up to date for ankane/pgvector:latest
docker.io/ankane/pgvector:latest
docker: Error response from daemon: Conflict. The container name "/pgvector-eccenca" is already in use by container "b6c885e9ba8fed6a52c3cfd6369cb2ba3df7c519423b50b647811974d1367bbb". You have to remove (or rename) that container to be able to reuse that name.

Run 'docker run --help' for more information


In [5]:
from IPython.display import Markdown
from llama_index.core import VectorStoreIndex
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.response_synthesizers import CompactAndRefine
from llama_index.core.retrievers import QueryFusionRetriever
from llama_index.core.retrievers.fusion_retriever import FUSION_MODES
from llama_index.vector_stores.postgres import PGVectorStore

hybrid_vector_store = PGVectorStore.from_params(
    database="eccenca",
    host="localhost",
    password="eccenca",
    port="5432",
    user="eccenca",
    table_name="prod-inst_hybrid",
    embed_dim=1536,  # openai embedding dimension
    hybrid_search=True,
    hnsw_kwargs={
        "hnsw_m": 16,
        "hnsw_ef_construction": 64,
        "hnsw_ef_search": 40,
        "hnsw_dist_method": "vector_cosine_ops",
    },
)

hybrid_index = VectorStoreIndex.from_vector_store(vector_store=hybrid_vector_store)

vector_retriever = hybrid_index.as_retriever(
    vector_store_query_mode="default",
    similarity_top_k=30,
)
text_retriever = hybrid_index.as_retriever(
    vector_store_query_mode="sparse",
    sparse_top_k=30,
)
retriever = QueryFusionRetriever(
    [vector_retriever, text_retriever],
    similarity_top_k=30,
    num_queries=1,  # set this to 1 to disable query generation
    mode=FUSION_MODES.RELATIVE_SCORE,
    use_async=False,
)

response_synthesizer = CompactAndRefine()
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

question = """What can you tell me about Dietlinde Boehme? \
Give me as many information and details as possible. \
Format the result nicely as Markdown. \
If sensible use list-style format. \
Also add a textual summary."""
answer = query_engine.query(question)

display(Markdown(f"### Question: _{question}_"))
display(Markdown(f"### Answer:\n\n{answer}"))
display(Markdown("#### Source nodes:"))
for node in answer.source_nodes:
    display(Markdown(f"> __Text:__ {node.text}\n>\n> __Score:__ {node.score}"))

PG Setup: Error creating HNSW index: (psycopg2.errors.SyntaxError) syntax error at or near "-"
LINE 1: CREATE INDEX IF NOT EXISTS data_prod-inst_hybrid_embedding_i...
                                            ^

[SQL: CREATE INDEX IF NOT EXISTS data_prod-inst_hybrid_embedding_idx ON public.data_prod-inst_hybrid USING hnsw (embedding vector_cosine_ops) WITH (m = 16, ef_construction = 64)]
(Background on this error at: https://sqlalche.me/e/20/f405)


### Question: _What can you tell me about Dietlinde Boehme? Give me as many information and details as possible. Format the result nicely as Markdown. If sensible use list-style format. Also add a textual summary._

### Answer:

- Dietlinde Boehme is a member of the Marketing department.
- Liese Adam, Bert Blumstein, Sabrina Geiger, Henny Foth, Valda Everhart, Yanka Schreiber, Baldwin Dirksen, Lambert Faust, Kevin Feigenbaum all have Dietlinde Boehme as their manager in the Marketing department.
- Dietlinde Boehme is the product manager for various products with prices ranging from 0,42 EUR to 4,49 EUR.
- Dietlinde Boehme manages the product managers of different hardware products such as Crystal Strain, Planck Dipole Driver Crystal, Capacitor Warp, Transformer Inductor Encoder, LCD Potentiometer Transducer, Resonator Strain Meter, Field-effect Network Rheostat, Network Multiplexer, Breaker Transducer, Strain Encoder, Log-periodic Driver Compensator, Coil Oscillator.
- Dietlinde Boehme is not part of the Product Management department.
- Dietlinde Boehme does not have any employees under her management in the Product Management department.
- Dietlinde Boehme does not have any specific area of expertise mentioned in the provided context.

### Summary:
Dietlinde Boehme is a key figure in the Marketing department, managing a team of employees and overseeing a range of products as their product manager. She plays a crucial role in the organization's product management and marketing efforts.

#### Source nodes:

> __Text:__ Dietlinde Boehme member of Marketing
>
> __Score:__ 1.0

...

> __Text:__ Dieterich Blau member of Product Management
>
> __Score:__ 0.07886477706878789

> __Text:__ Dieterich Blau has manager Franz Kornhaeusel
>
> __Score:__ 0.06584489133707765

> __Text:__ Dieterich Blau area of expertise Breaker
>
> __Score:__ 0.044020135924568486

> __Text:__ Dieterich Blau area of expertise Resistor
>
> __Score:__ 0.02828604100123162

> __Text:__ Berlin Schulz has manager Franz Kornhaeusel
>
> __Score:__ 0.0